In [1]:
# 8.5.2_replacement_analysis_finite_horizon_Dynamic_Programming.ipynb
""" 8.5.2 Optimal Replacement under Finite Planning Horizon
    Dynamic Programming Approach """
import pandas as pd
import numpy_financial as npf
from EngFinancialPy import Asset, pprint_list

In [2]:
# Global constants
marr = 0.1
study_period = 6

In [3]:
# Global data structures for optimal solution tracing
data = pd.DataFrame() # Assets data
decision = {}  # Decisions made at all nodes visited
opt_path = {}  # Optimnal decision path traced

In [4]:
def main():
    print("\nOptimal Replacement, Finite Study Period using Dynamic Programming")
    print(f"Study period = {study_period} years")
    print(f"marr = {marr*100:.1f} %")
    
    Assets_Data()
    print(f"\nAvailable assets: {list(data.index)}")
    
    # initial states for Dynamic Programming
    year_now = 0
    current_asset = data.index[0]
    used = 0
    
    # Compute optimal solution using dynamic programming
    EPC_opp = DP(year_now, current_asset, used)

    # Trace the optimal decsions for each year
    Trace_optimal_solution(year_now, current_asset, used)
    print("\nOptimal Decisions:")
    for s in opt_path:
        print(f"  {s} -> {opt_path[s]}")
        
    # Display years of usage of each asset (k1, k2, k3)
    print("\nOptimal replacement plan:",
             [data["usage"][a] for a in data.index])
    for a in data.index:
        print(f"  Use {a} for {data['usage'][a]} years")
    
    # Display Optimal EPC and EUAC
    print(f"\nOptimal EPC (opportunity) = {EPC_opp:10,.2f}")
    EPC_cf = EPC_opp - data['MV0']['D0']
    print(f"Optimal EPC (cash flows)  = {EPC_cf:10,.2f}")
    EUAC_cf = -npf.pmt(marr, study_period, EPC_cf, 0)
    print(f"Optimal EUAC (cash flows) = {EUAC_cf:10,.2f}")

In [5]:
# Set up data
def Assets_Data():
    global data
    
    # Defender data
    MV0_d = 5000
    MV_d = [4000, 3000, 2000, 1000 ]  
    E_d = [5500, 6600, 7800, 8800 ]
    defender = Asset(MV0_d, MV_d, E_d, marr, name="Defender")
    TC_d = defender.TC()
    life_d = defender.useful_life()
    print(f"Defender remaining life = {life_d} years")
    pprint("Defender TC", TC_d)
    
    # Challenger data
    MV0_c = 20000
    MV_c = [15000, 11250, 8500, 6500,  4750 ]  
    E_c = [ 2000,  3000, 4620, 8000, 12000 ]
    challenger = Asset(MV0_c, MV_c, E_c, marr, age=0, name="Challenger")
    TC_c = challenger.TC()
    life_c = challenger.useful_life()
    print(f"Challenger useful life = {life_c} years")
    pprint("Challenger TC", TC_c)
    econ_life_c, euac_star = challenger.econ_life_euac()
    print(f"Challenger econ life = {econ_life_c} yrs at EUAC*={euac_star:,.2f}") 
    
    # Put all available assets into a dictionary
    assets_data = { 'asset' : ["D0",   "C1",    "C2" ],
                    'TC'    : [ TC_d,   TC_c,   TC_c ],
                    'MV0'   : [ MV0_d,  MV0_c,  MV0_c],
                    'life'  : [ life_d, life_c, life_c],
                    'usage' : [ 0, 0, 0] }  # starting (k1, k2, k3)

    # Convert assets data to dataframe
    data = pd.DataFrame(assets_data)
    data = data.set_index('asset')

In [6]:
# next available asset for replacement if any
def next_asset(mc):
    """ Determine next available asset for replacement if any """
    assets = list(data.index)
    head = assets.pop(0)
    while assets != []:
        if head == mc : 
            return assets[0]
        else: 
            head = assets.pop(0)
    return 'Nil'

In [7]:
## Dynamic Programming 
def DP(t, mc, used):
    """ Perform Dynamic Programming """
    global decision
    # Study period reached
    if t == study_period:
        decision[(t,mc,used)] = "End"
        return 0
    nex = next_asset(mc)
    if used == data['life'][mc] and nex == "Nil":
        # Current asset reached max life and no replacement is available
        decision[(t,mc,used)] = "Infeasible"
        return 10**10  # Big-M
    if used == data['life'][mc] and nex != "Nil":
        # Current asset reached max life and a replacement is available
        # Replace only
        replace_cost = data['TC'][nex][0]/(1+marr)**(t+1) + DP(t+1,nex,1)
        decision[(t,mc,used)] = "Replace"
        return replace_cost
    if used < data['life'][mc] and nex == "Nil":
        # Current asset has not reached max life and no replacement 
        # is available. Therefore, Keep only
        keep_cost = data['TC'][mc][used]/(1+marr)**(t+1) + DP(t+1,mc,used+1)
        decision[(t,mc,used)] = "Keep"
        return keep_cost
    if used < data['life'][mc] and nex != "Nil":
        # Current asset has not reached max life and a replacement is available
        # Choice of either Keep or Replace
        replace_cost = data['TC'][nex][0]/(1+marr)**(t+1) + DP(t+1,nex,1)
        keep_cost = data['TC'][mc][used]/(1+marr)**(t+1) + DP(t+1,mc,used+1)
        if keep_cost < replace_cost:
            decision[(t,mc,used)] = "Keep"
            return keep_cost
        else:
            decision[(t,mc,used)] = "Replace"
            return replace_cost

In [8]:
# Trace the optimal replacement decisions
def Trace_optimal_solution(t, mc, used):
    """ Trace Optimal Replacement Decisions """
    global data
    global opt_path
    state = (t, mc, used)
    action = decision[state]
    opt_path[state] = action
    # print(state, action)
    if action == "End": return
    if action == "Infeasible": return
    if action == "Keep":
        data.loc[mc, 'usage'] += 1
        Trace_optimal_solution(t+1, mc, used+1)
    if action == "Replace":
        data.loc[next_asset(mc), 'usage'] += 1
        Trace_optimal_solution(t+1, next_asset(mc), 1)

In [9]:
def pprint(label, list_of_numbers):
    """ Pretty format print a List of numbers """
    print(f"{label} =", "".join(f'{x:,.2f}   ' for x in list_of_numbers))

In [10]:
main()


Optimal Replacement, Finite Study Period using Dynamic Programming
Study period = 6 years
marr = 10.0 %
Defender remaining life = 4 years
Defender TC = 7,000.00   8,000.00   9,100.00   10,000.00   
Challenger useful life = 5 years
Challenger TC = 9,000.00   8,250.00   8,495.00   10,850.00   14,400.00   
Challenger econ life = 3 yrs at EUAC*=8,598.19

Available assets: ['D0', 'C1', 'C2']

Optimal Decisions:
  (0, 'D0', 0) -> Keep
  (1, 'D0', 1) -> Keep
  (2, 'D0', 2) -> Replace
  (3, 'C1', 1) -> Keep
  (4, 'C1', 2) -> Replace
  (5, 'C2', 1) -> Keep
  (6, 'C2', 2) -> End

Optimal replacement plan: [2, 2, 2]
  Use D0 for 2 years
  Use C1 for 2 years
  Use C2 for 2 years

Optimal EPC (opportunity) =  35,617.10
Optimal EPC (cash flows)  =  30,617.10
Optimal EUAC (cash flows) =   7,029.91
